In [1]:
import time
import warnings

import pandas as pd
from sklearn.datasets import make_regression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from histree import HistogramRandomForestRegressor

warnings.filterwarnings('ignore')

N_SAMPLES = 1_000_000
N_FEATURES = 50
NOISE = 10
TEST_SIZE = 200_000

print(f"\nGenerowanie danych ({N_SAMPLES} próbek, {N_FEATURES} cech)...")
X, y = make_regression(n_samples=N_SAMPLES, n_features=N_FEATURES, noise=NOISE, random_state=42)

X_train = X[:-TEST_SIZE]
y_train = y[:-TEST_SIZE]
X_test = X[-TEST_SIZE:]
y_test = y[-TEST_SIZE:]

print(f"Dane gotowe: Train={len(X_train)}, Test={len(X_test)}")

results = []

def run_model(name, model):
    print(f"\n--- Trenowanie: {name} ---")
    start = time.time()
    model.fit(X_train, y_train)
    t_fit = time.time() - start
    print(f"   -> Czas treningu: {t_fit:.4f} s")
    start = time.time()
    preds = model.predict(X_test)
    t_pred = time.time() - start
    print(f"   -> Czas predykcji: {t_pred:.4f} s")
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return {
        "Model": name,
        "Train Time (s)": t_fit,
        "Predict Time (s)": t_pred,
        "RMSE": rmse,
        "R2 Score": r2
    }

sk_rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=12,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features=0.9,
    n_jobs=-1,
    random_state=2137,
    verbose=0
)
results.append(run_model("RandomForest (sklearn)", sk_rf))

my_rf = HistogramRandomForestRegressor(
    n_estimators=100,
    max_depth=12,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features=0.9,
    n_bins=64,
    bootstrap=True,
    min_gain_to_split=0.0,
    n_jobs=-1,
    random_state=2137,
)
results.append(run_model("CustomRF (Python/Numba)", my_rf))

df_res = pd.DataFrame(results)

print("\n" + "="*60)
print("             🏆 WYNIKI BENCHMARKU 🏆")
print("="*60)
print(df_res.to_markdown(index=False, floatfmt=".4f"))
print("\n" + "-"*60)
print("ANALIZA:")
for i, row in df_res.iterrows():
    print(f"{row['Model']}: RMSE={row['RMSE']:.4f}, R2={row['R2 Score']:.4f}, Train={row['Train Time (s)']:.2f}s, Predict={row['Predict Time (s)']:.2f}s")
print("-"*60)


Generowanie danych (1000000 próbek, 50 cech)...
Dane gotowe: Train=800000, Test=200000

--- Trenowanie: RandomForest (sklearn) ---
   -> Czas treningu: 444.0804 s
   -> Czas predykcji: 0.2795 s

--- Trenowanie: CustomRF (Python/Numba) ---
   -> Czas treningu: 20.4994 s
   -> Czas predykcji: 0.3012 s

             🏆 WYNIKI BENCHMARKU 🏆
| Model                   |   Train Time (s) |   Predict Time (s) |    RMSE |   R2 Score |
|:------------------------|-----------------:|-------------------:|--------:|-----------:|
| RandomForest (sklearn)  |         444.0804 |             0.2795 | 45.2757 |     0.8973 |
| CustomRF (Python/Numba) |          20.4994 |             0.3012 | 44.3288 |     0.9016 |

------------------------------------------------------------
ANALIZA:
RandomForest (sklearn): RMSE=45.2757, R2=0.8973, Train=444.08s, Predict=0.28s
CustomRF (Python/Numba): RMSE=44.3288, R2=0.9016, Train=20.50s, Predict=0.30s
------------------------------------------------------------


In [3]:
import time
import warnings

import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor
from histree import HistogramGradientBoostingRegressor

warnings.filterwarnings('ignore')

N_SAMPLES = 1_000_000
N_FEATURES = 50
NOISE = 10
TEST_SIZE = 200_000

print(f"\nGenerowanie danych ({N_SAMPLES} próbek, {N_FEATURES} cech)...")
X, y = make_regression(n_samples=N_SAMPLES, n_features=N_FEATURES, noise=NOISE, random_state=42)

X_train = X[:-TEST_SIZE]
y_train = y[:-TEST_SIZE]
X_test = X[-TEST_SIZE:]
y_test = y[-TEST_SIZE:]

print(f"Dane gotowe: Train={len(X_train)}, Test={len(X_test)}")

results = []

def run_model(name, model):
    print(f"\n--- Trenowanie: {name} ---")
    start = time.time()
    model.fit(X_train, y_train)
    t_fit = time.time() - start
    print(f"   -> Czas treningu: {t_fit:.4f} s")
    start = time.time()
    preds = model.predict(X_test)
    t_pred = time.time() - start
    print(f"   -> Czas predykcji: {t_pred:.4f} s")
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return {
        "Model": name,
        "Train Time (s)": t_fit,
        "Predict Time (s)": t_pred,
        "RMSE": rmse,
        "R2 Score": r2
    }

lgbm = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=64, 
    max_depth=6,
    min_child_samples=20,
    reg_lambda=1.0,
    n_jobs=-1,
    random_state=2137,
    verbose=-1
)
results.append(run_model("LightGBM (C++)", lgbm))

sk_histgb = HistGradientBoostingRegressor(
    max_iter=100,
    learning_rate=0.1,
    max_depth=6,
    min_samples_leaf=20,
    l2_regularization=1.0,
    max_bins=255,
    random_state=2137,
    verbose=0
)
results.append(run_model("HistGB (sklearn)", sk_histgb))

my_model = HistogramGradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    min_samples_leaf=20,
    reg_lambda=1.0,
    n_bins=255,
    random_state=2137
)
results.append(run_model("CustomGB (Python/Numba)", my_model))

df_res = pd.DataFrame(results)

print("\n" + "="*60)
print("             🏆 WYNIKI BENCHMARKU 🏆")
print("="*60)
print(df_res.to_markdown(index=False, floatfmt=".4f"))
print("\n" + "-"*60)
print("ANALIZA:")
lgbm_row = df_res[df_res["Model"] == "LightGBM (C++)"].iloc[0]
sk_row = df_res[df_res["Model"] == "HistGB (sklearn)"].iloc[0]
custom_row = df_res[df_res["Model"] == "CustomGB (Python/Numba)"].iloc[0]
print(f"1. Czas Treningu: Twój model jest {custom_row['Train Time (s)']/lgbm_row['Train Time (s)']:.2f}x wolniejszy niż LightGBM, {custom_row['Train Time (s)']/sk_row['Train Time (s)']:.2f}x wolniejszy niż HistGB.")
print(f"2. Czas Predykcji: Twój model jest {custom_row['Predict Time (s)']/lgbm_row['Predict Time (s)']:.2f}x wolniejszy niż LightGBM, {custom_row['Predict Time (s)']/sk_row['Predict Time (s)']:.2f}x wolniejszy niż HistGB.")
print(f"3. Dokładność: Twój RMSE jest gorszy o {custom_row['RMSE']-lgbm_row['RMSE']:.4f} względem LightGBM, o {custom_row['RMSE']-sk_row['RMSE']:.4f} względem HistGB.")
print("-"*60)


Generowanie danych (1000000 próbek, 50 cech)...
Dane gotowe: Train=800000, Test=200000

--- Trenowanie: LightGBM (C++) ---
   -> Czas treningu: 3.9689 s
   -> Czas predykcji: 0.1165 s

--- Trenowanie: HistGB (sklearn) ---
   -> Czas treningu: 5.4684 s
   -> Czas predykcji: 0.2380 s

--- Trenowanie: CustomGB (Python/Numba) ---
   -> Czas treningu: 5.0127 s
   -> Czas predykcji: 0.1081 s

             🏆 WYNIKI BENCHMARKU 🏆
| Model                   |   Train Time (s) |   Predict Time (s) |    RMSE |   R2 Score |
|:------------------------|-----------------:|-------------------:|--------:|-----------:|
| LightGBM (C++)          |           3.9689 |             0.1165 | 15.3106 |     0.9883 |
| HistGB (sklearn)        |           5.4684 |             0.2380 | 15.8335 |     0.9874 |
| CustomGB (Python/Numba) |           5.0127 |             0.1081 | 18.6277 |     0.9826 |

------------------------------------------------------------
ANALIZA:
1. Czas Treningu: Twój model jest 1.26x wolniejs

In [4]:
import time
import warnings

import pandas as pd
from sklearn.datasets import make_regression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostRegressor
from histree import HistogramSequentialWeightedForestRegressor

warnings.filterwarnings('ignore')

N_SAMPLES = 1_000_000
N_FEATURES = 50
NOISE = 10
TEST_SIZE = 200_000

print(f"\nGenerowanie danych ({N_SAMPLES} próbek, {N_FEATURES} cech)...")
X, y = make_regression(n_samples=N_SAMPLES, n_features=N_FEATURES, noise=NOISE, random_state=42)

X_train = X[:-TEST_SIZE]
y_train = y[:-TEST_SIZE]
X_test = X[-TEST_SIZE:]
y_test = y[-TEST_SIZE:]

print(f"Dane gotowe: Train={len(X_train)}, Test={len(X_test)}")

results = []

def run_model(name, model):
    print(f"\n--- Trenowanie: {name} ---")
    start = time.time()
    model.fit(X_train, y_train)
    t_fit = time.time() - start
    print(f"   -> Czas treningu: {t_fit:.4f} s")
    start = time.time()
    preds = model.predict(X_test)
    t_pred = time.time() - start
    print(f"   -> Czas predykcji: {t_pred:.4f} s")
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return {
        "Model": name,
        "Train Time (s)": t_fit,
        "Predict Time (s)": t_pred,
        "RMSE": rmse,
        "R2 Score": r2
    }

ada = AdaBoostRegressor(
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)
results.append(run_model("AdaBoostRegressor (sklearn)", ada))

my_swf = HistogramSequentialWeightedForestRegressor(
    n_estimators=100,
    learning_rate=1.0,
    max_depth=6,
    min_samples_split=20,
    min_samples_leaf=10,
    reg_lambda=1.0,
    min_gain_to_split=0.0,
    n_bins=255,
    random_state=2137,
)
results.append(run_model("CustomSWF (Python/Numba)", my_swf))

df_res = pd.DataFrame(results)

print("\n" + "="*60)
print("             🏆 WYNIKI BENCHMARKU 🏆")
print("="*60)
print(df_res.to_markdown(index=False, floatfmt=".4f"))
print("\n" + "-"*60)
print("ANALIZA:")
for i, row in df_res.iterrows():
    print(f"{row['Model']}: RMSE={row['RMSE']:.4f}, R2={row['R2 Score']:.4f}, Train={row['Train Time (s)']:.2f}s, Predict={row['Predict Time (s)']:.2f}s")
print("-"*60)


Generowanie danych (1000000 próbek, 50 cech)...
Dane gotowe: Train=800000, Test=200000

--- Trenowanie: AdaBoostRegressor (sklearn) ---
   -> Czas treningu: 568.9444 s
   -> Czas predykcji: 1.3064 s

--- Trenowanie: CustomSWF (Python/Numba) ---
   -> Czas treningu: 7.6463 s
   -> Czas predykcji: 0.2143 s

             🏆 WYNIKI BENCHMARKU 🏆
| Model                       |   Train Time (s) |   Predict Time (s) |    RMSE |   R2 Score |
|:----------------------------|-----------------:|-------------------:|--------:|-----------:|
| AdaBoostRegressor (sklearn) |         568.9444 |             1.3064 | 43.5958 |     0.9048 |
| CustomSWF (Python/Numba)    |           7.6463 |             0.2143 | 31.6493 |     0.9498 |

------------------------------------------------------------
ANALIZA:
AdaBoostRegressor (sklearn): RMSE=43.5958, R2=0.9048, Train=568.94s, Predict=1.31s
CustomSWF (Python/Numba): RMSE=31.6493, R2=0.9498, Train=7.65s, Predict=0.21s
---------------------------------------------